In [7]:
import numpy as np
import librosa, librosa.display
import matplotlib.pyplot as plt
import math
import os
import json
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


In [8]:
def load_data(data_path):

    with open(data_path, "r") as f:
        data = json.load(f)

    # convert lists to numpy arrays
    X = np.array(data["mfcc"])
    y = np.array(data["labels"])

    print("Data succesfully loaded!")

    return  X, y

In [9]:
# load data
X, y = load_data("/content/drive/MyDrive/dataset/genres/data_10 (1).json")

Data succesfully loaded!


In [10]:

X.shape

(9996, 130, 13)

In [11]:
# create train, validation and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2)

# add an axis to input sets
X_train = X_train[..., np.newaxis]
X_validation = X_validation[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [12]:

X_train.shape

(5997, 130, 13, 1)

In [13]:

input_shape = (X_train.shape[1], X_train.shape[2], 1)

In [14]:
# build the CNN
model_cnn = keras.Sequential()

# 1st conv layer
model_cnn.add(keras.layers.Conv2D(40, (3, 3), activation='relu', input_shape=input_shape))
model_cnn.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
model_cnn.add(keras.layers.BatchNormalization())

# 2nd conv layer
model_cnn.add(keras.layers.Conv2D(40, (3, 3), activation='relu'))
model_cnn.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
model_cnn.add(keras.layers.BatchNormalization())


# 3rd conv layer
model_cnn.add(keras.layers.Conv2D(40, (2, 2), activation='relu'))
model_cnn.add(keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
model_cnn.add(keras.layers.BatchNormalization())



# flatten output and feed it into dense layer
model_cnn.add(keras.layers.Flatten())
model_cnn.add(keras.layers.Dense(70, activation='relu'))
model_cnn.add(keras.layers.Dropout(0.3))

# output layer
model_cnn.add(keras.layers.Dense(15, activation='softmax'))

In [15]:
# compile model
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model_cnn.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
model_cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 11, 40)       400       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 6, 40)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 6, 40)         160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 62, 4, 40)         14440     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 2, 40)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 31, 2, 40)         160       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 1, 40)         6

In [17]:
# train model
history = model_cnn.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=50)

Epoch 1/50
188/188 [==============================] - 16s 80ms/step - loss: 2.6019 - accuracy: 0.2158 - val_loss: 1.9084 - val_accuracy: 0.3747
Epoch 2/50
188/188 [==============================] - 15s 79ms/step - loss: 1.8822 - accuracy: 0.3784 - val_loss: 1.5150 - val_accuracy: 0.4740
Epoch 3/50
188/188 [==============================] - 15s 80ms/step - loss: 1.6347 - accuracy: 0.4457 - val_loss: 1.3523 - val_accuracy: 0.5173
Epoch 4/50
188/188 [==============================] - 15s 80ms/step - loss: 1.4820 - accuracy: 0.4876 - val_loss: 1.2563 - val_accuracy: 0.5553
Epoch 5/50
188/188 [==============================] - 17s 90ms/step - loss: 1.3906 - accuracy: 0.5194 - val_loss: 1.1940 - val_accuracy: 0.5900
Epoch 6/50
188/188 [==============================] - 15s 80ms/step - loss: 1.2961 - accuracy: 0.5596 - val_loss: 1.1398 - val_accuracy: 0.5980
Epoch 7/50
188/188 [==============================] - 15s 79ms/step - loss: 1.2611 - accuracy: 0.5641 - val_loss: 1.1188 - val_accuracy:

In [18]:
# evaluate model on Test Set
test_loss, test_acc = model_cnn.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

79/79 - 1s - loss: 0.7424 - accuracy: 0.7579

Test accuracy: 0.7579031586647034


In [24]:
# evaluate model on Test Set
test_loss, test_precision = model_cnn.evaluate(X_test, y_test, verbose=2)
print('\nTest Precision:', test_precision)

79/79 - 1s - loss: 0.7424 - accuracy: 0.7579

Test Precision: 0.7579031586647034


In [25]:
# evaluate model on Test Set
test_loss, test_recall = model_cnn.evaluate(X_test, y_test, verbose=2)
print('\nTest Recall:',test_recall)

79/79 - 1s - loss: 0.7424 - accuracy: 0.7579

Test Recall: 0.7579031586647034


In [27]:
# evaluate model on Test Set
test_loss, test_f1score = model_cnn.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_f1score)

79/79 - 1s - loss: 0.7424 - accuracy: 0.7579

Test accuracy: 0.7579031586647034


In [19]:

model_cnn.save("Music_Genre_10_CNN")

INFO:tensorflow:Assets written to: Music_Genre_10_CNN/assets
